In [19]:
#Imports
from math import log, sqrt, pi, exp
from scipy.stats import norm
import pandas as pd
import tensorflow as tf
import numpy as np

def read_file(file):
    """Read a single file and return a dataframe"""
    return pd.read_csv(file, skipinitialspace=True)

In [20]:
# Black-Scholes formula for call options
def d1(S,K,T,r,sigma):
    x1 = S.apply(lambda x : log(x)) - K.apply(lambda x : log(x))
    x2 = (r + ((sigma.apply(lambda x : x**2)) / 2)) * T
    x3 = sigma * T.apply(lambda x: sqrt(x))
    return  (x1 + x2) / x3

def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma) - sigma * T.apply(lambda x : sqrt(x))  

def bs_call(S,K,T,r,sigma):
    T = T/365
    r = r/100
    return S * d1(S,K,T,r,sigma).apply(lambda x : norm.cdf(x)) - K * (-r*T).apply(lambda x : exp(x)) * d2(S,K,T,r,sigma).apply(lambda x : norm.cdf(x))

In [21]:
#Insert file with LSTM predictions
df_options = read_file("data/2021_december_predictions_11-24_13-50.csv")
#Calculating BS and add column
df_options["B-S"] = bs_call(df_options["Underlying_last"], df_options["Strike"], df_options["Ttl"], df_options["R"], df_options["Volatility"])
#Cleaning
df_options = df_options[["Quote_date", "Underlying_last", "Strike", "Ttl", "Volatility", "R", "Price", "B-S", "Prediction"]]
df_options = df_options.rename(columns={"Prediction":"LSTM"})

Creating table

In [22]:
df_low_money = df_options[df_options["Underlying_last"] / df_options["Strike"] < 0.97]
df_low_money.name = "<0.97"
df_mid_money = df_options[(df_options["Underlying_last"] / df_options["Strike"]).between(0.97, 1.03, inclusive=True)]
df_mid_money.name = "0.97-1.03"
df_high_money = df_options[df_options["Underlying_last"] / df_options["Strike"] > 1.03]
df_high_money.name = ">1.03"

In [24]:
df_RSME = pd.DataFrame(columns=['Moneyness', 'Maturity', 'BS','LSTM'])
for dataframex in [df_low_money, df_mid_money, df_high_money]:
    i = 1
    for days in [89,90,90,180,180,455]:
        df_temp = dataframex[(df_options["Ttl"]).between(i, i + days, inclusive=True)]
        bs = sqrt(tf.keras.metrics.mean_squared_error(df_temp["Price"], df_temp["B-S"]))
        lstm = sqrt(tf.keras.metrics.mean_squared_error(df_temp["Price"], df_temp["LSTM"]))
        df_RSME.loc[len(df_RSME.index)] = [dataframex.name, str(i) + "-" + str(i + days), bs, lstm]
        i = i + days

<ipython-input-24-5769fde4d2ba>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + days, inclusive=True)]
<ipython-input-24-5769fde4d2ba>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + days, inclusive=True)]
<ipython-input-24-5769fde4d2ba>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + days, inclusive=True)]
<ipython-input-24-5769fde4d2ba>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + days, inclusive=True)]
<ipython-input-24-5769fde4d2ba>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + days, inclusive=True)]
<ipython-input-24-5769fde4d2ba>:5: UserW

Writing model

In [25]:
with open('december_results_RSME.tex', 'w') as tf:
     tf.write(df_RSME.to_latex(index=False))